# 3. PHD Filtering

In [1]:
from matplotlib import pyplot as plt
import numpy as np
from datetime import datetime
from phdsupplement import gen_ground_truths, gen_detectors, run_tracker

Let us generate some paths using the following parameters

In [2]:
# Parameters
start_time = datetime.now()
birth_probability = 0.2
death_probability = 0.005
number_steps = 20

truths, truths_by_time, start_truths, transition_model = gen_ground_truths(start_time, birth_probability, death_probability, number_steps)

# Then generate path detections
detection_probability = 0.9
clutter_rate = 3.0
all_measurements, measurement_model = gen_detectors(truths, start_time, number_steps, detection_probability, clutter_rate=clutter_rate)

# Run tracker
tracks = run_tracker(all_measurements, measurement_model, transition_model, detection_probability, death_probability, clutter_rate, start_truths, start_time)

Plotting these values give

In [3]:
from stonesoup.plotter import Plotterly
plotter = Plotterly()
plotter.plot_ground_truths(truths, [0, 2])
plotter.fig

Plotting the measurements (including clutter)

In [4]:
# Plot true detections and clutter.
plotter.plot_measurements(all_measurements, [0, 2])
plotter.fig

Plot the tracks

In [5]:
from stonesoup.types.detection import TrueDetection
x_min, x_max, y_min, y_max = 0, 0, 0, 0

# Get bounds from the tracks
for track in tracks:
    for state in track:
        x_min = min([state.state_vector[0], x_min])
        x_max = max([state.state_vector[0], x_max])
        y_min = min([state.state_vector[2], y_min])
        y_max = max([state.state_vector[2], y_max])

# Get bounds from measurements
for measurement_set in all_measurements:
    for measurement in measurement_set:
        if type(measurement) == TrueDetection:
            x_min = min([measurement.state_vector[0], x_min])
            x_max = max([measurement.state_vector[0], x_max])
            y_min = min([measurement.state_vector[1], y_min])
            y_max = max([measurement.state_vector[1], y_max])

# Plot the tracks
plotter = Plotterly()
plotter.plot_ground_truths(truths, [0, 2])
plotter.plot_measurements(all_measurements, [0, 2])
plotter.plot_tracks(tracks, [0, 2], uncertainty=True)
plotter.fig.update_xaxes(range=[x_min-5, x_max+5])
plotter.fig.update_yaxes(range=[y_min-5, y_max+5])
plotter.fig

With a high detection probability, the PHD filter is able to resolve the ground truths much more accurately. This is due to the fact that the  is higher which means the posterior density function